### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# sample num
sending_sample_num = 400  # 150,200,300,500

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
##cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
#app = firebase_admin.initialize_app(cred)

In [7]:
#fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(sending_sample_num)
df_28

,video_id,start_time,end_time,text
0,IGJVw1HjIHc,4266,4269,yok çok değil o kadar da değil
1,YedJhuZQqpI,6690,6693,hayır hayır bir ama ben sen
2,pvsImT6E4yM,6351,6354,ama bu kadar bir şey yok da
3,FzHO9BqY_Vc,50,53,yok yok ben de bir şey yok
4,4LtEd0U7rOw,297,299,yok bu değil de o da değil
...,...,...,...,...
123,KgFHQZj1CLw,99,100,bana mı var
124,WspUg4WmNHI,232,234,ama ben var
125,cslXqrcH5qg,3896,3897,ne kadar bu
126,eQZssVr1Y64,1300,1302,ben de evet


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(sending_sample_num)
df_200

,video_id,start_time,end_time,text
0,ZdeFof81xKs,720,725,bilmiyorum sen ne demek istiyorsun ben de onu ...
1,zgQbOmPp9pk,7920,7926,var da bu bir sorun olur mu benim için ben bun...
2,f8iA6v4mRAY,394,400,o çok güzel oldu gerçekten herkes gibi oldu ge...
3,GZG9nDbIgXM,1827,1831,evet ben tabii onları bilmiyorum onlar daha şe...
4,MvUL_IHyESU,17,23,olarak da çok daha büyük şimdi buna bakalım ge...
...,...,...,...,...
183,XnakBVeh0n4,1891,1893,ya bilmiyorum çok fazla mı ben böyle
184,h57En60xdPk,411,415,kadar buraya geldi daha sonra tekrar
185,9NguZ1ukR5E,403,408,böyle şeyler çünkü bilmiyorum bence
186,6TOnCfxXzyU,1173,1175,tabii ne kadar doğru bilmiyorum ama


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(sending_sample_num)
df_1000

,video_id,start_time,end_time,text
0,KsZ_8oIQsv4,318,325,başka biriyle herhangi bir şey yaşıyor musun k...
1,cUAQrHbbpZ8,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...
2,Mh3bgDLKt2o,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...
3,sHnvEDmE3so,272,277,benim burada ne işim var neden bunu yapıyorum ...
4,zgQbOmPp9pk,5380,5389,bu işi yaparım dedim o zaman süper dedi bitti ...
...,...,...,...,...
230,h93wXNLXmYU,898,901,şimdi diyorum ki söz verdim insanlar beni bekl...
231,V9n8LpTPcfM,44,47,beni hep al yani ona seviyorum tamam tamam o g...
232,ry0hEicwuhQ,852,856,en yakın arkadaşım dedi ki senin kızın olacak ...
233,nYNwFO8hA2g,56,60,daha kolay ve hızlı ilk olarak bunu kabul ediy...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(sending_sample_num)
df_5000

,video_id,start_time,end_time,text
0,MQ_nenSkjbo,380,395,şimdi ben seni tabii hem kişisel olarak hem de...
1,dhV8ToP5VTQ,399,407,ama ben kimseye durup da spor yapmak isteyen i...
2,SgYj1SsG1LQ,1638,1647,daha ciddi bir şey hak ettiğini biliyor ve bun...
3,3mpFwkjir0A,1629,1637,böyle karmaşık birileri hava atıyor biri karı ...
4,GimA7_uhyWg,811,820,ve bir şansın olsa belki de bazı şeyleri daha ...
...,...,...,...,...
395,f8iA6v4mRAY,1369,1378,arkadaşlarım tatlı evet şimdi sıra geldi şu pa...
396,Mh8BTIbj5K8,910,916,sevgili dostlar burada başarısız olduğunuz maa...
397,kSlAhrlx1lw,2369,2376,biz gidiyoruz ama gitme diyor bize ya burası r...
398,L4ed7PRsvxE,613,619,gelirim herhalde buraya devamlı bakalım daha b...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(sending_sample_num)
df_10000

,video_id,start_time,end_time,text
0,LPrMSJHDDcA,678,690,on tane geliyor haftada üç dört kere şeye çıkı...
1,FIcnpr3UtaQ,696,705,okulun ilk günü ne yapacağımızı bilmiyorduk bi...
2,MQ_nenSkjbo,380,395,şimdi ben seni tabii hem kişisel olarak hem de...
3,ywf4HpQKuf0,110,119,o yüzden aslında hep sosyal medya ile bir şeki...
4,nB0Q5v1_kUs,362,371,ve bu gözle gelecek olan ikinci karakteri ki ö...
...,...,...,...,...
395,AvfpnsqaWw4,895,901,istiyorsanız koyun en azından neden olduğunu b...
396,_Xtfh7OGcGM,231,238,olan yer ya orası bayağı yani insanlar geliyor...
397,rMxxGQVkczU,892,899,en mutlu insanı olacağım babacığım seni sabah ...
398,TvLxPsIHAEk,5380,5386,kaynakları kişi o benim arkadaşım zaten berabe...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(sending_sample_num)
df_20000

,video_id,start_time,end_time,text
0,LOQOS5IJtlM,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...
1,aAa2PRgjkcg,238,250,yeni yılda eve şans gelmesi için gece yarısınd...
2,9eYN0BDyLFM,503,511,şahin bu sene üniversiteye başlayacağım ama di...
3,MQ_nenSkjbo,380,395,şimdi ben seni tabii hem kişisel olarak hem de...
4,it2FXAAbs88,758,767,aynısı aynısı ya çünkü niye sordum bana da çok...
...,...,...,...,...
395,jdpYInLk2Ow,1192,1197,içinde döner misin hocam hemen diyorum ki nedi...
396,mYdU9NOn2AY,231,237,alıyorum çünkü bunlar biraz daha kısa o yüzden...
397,gYoIugaLSJw,580,586,zaman kendini rahatsız huzursuz hissediyorsun ...
398,lG7qPNPsumU,41,47,için her türlü oyunu yapmaya hazırlar ne yazık...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(sending_sample_num)
df_40000

,video_id,start_time,end_time,text
0,CULrx3yerE8,409,425,baktığınızda aynı fikirde misiniz çoğunlukla h...
1,LPrMSJHDDcA,679,690,on tane geliyor haftada üç dört kere şeye çıkı...
2,aAa2PRgjkcg,238,251,yeni yılda eve şans gelmesi için gece yarısınd...
3,bwEEzF905Gk,587,595,bir sürü kızla çıkmış bir çocuk kızları düşünü...
4,LPrMSJHDDcA,333,345,cumartesi bildiğin iki saat aralıksız dört far...
...,...,...,...,...
395,ebVNk98rGzs,2195,2202,ne malzemeler kullandık hangi malzemelerin ark...
396,Fn4Fe0zm2xM,171,177,örneğin çocuğunuz dışarı çıkmak istiyor siz is...
397,_xfa4TEyUOo,366,372,fabrikada emir komuta tamamen katar ordusuna v...
398,1iByM08MvqU,1718,1724,anlayamadım telefonla ilgili bağlantıyı biraz ...


##### Calculate Speech Time Mean

In [16]:
(df_28["end_time"] - df_28["start_time"]).mean()

1.890625

In [17]:
(df_200["end_time"] - df_200["start_time"]).mean()

3.404255319148936

In [18]:
(df_1000["end_time"] - df_1000["start_time"]).mean()

5.3319148936170215

In [19]:
(df_5000["end_time"] - df_5000["start_time"]).mean()

7.3525

In [20]:
(df_10000["end_time"] - df_10000["start_time"]).mean()

7.9325

In [21]:
(df_20000["end_time"] - df_20000["start_time"]).mean()

8.09

In [22]:
(df_40000["end_time"] - df_40000["start_time"]).mean()

8.36

#### Sending Data To Firestore

In [23]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [24]:
fire_db = firestore.client()

##### Language Data

In [25]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [26]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [27]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [28]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [29]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [30]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [31]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [32]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [33]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [34]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [35]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [36]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [37]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [38]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [39]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [40]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [41]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [42]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [43]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [44]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [45]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [46]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [47]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [48]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [49]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [50]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [51]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [52]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [53]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [54]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [55]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [56]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [57]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1683640524
  nanos: 931072000
}